In [173]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

# Import XML outbound files from Roadnet

In [174]:
# Set filenames
consolidated_roadnet_out_file = 'data/roadnet/xml_consolidated/consolidated_roadnet_out.xml'
roadnet_inbound_file = 'data/roadnet/inbound/rdnet_in.csv'

In [175]:
try:
    os.remove(consolidated_roadnet_out_file)
except:
    print(consolidated_roadnet_out_file + ' not in folder')

try:
    os.remove(roadnet_inbound_file)
except:
    print(roadnet_inbound_file + ' not in folder')    

data/roadnet/xml_consolidated/consolidated_roadnet_out.xml not in folder


In [176]:
# Clean up the xml_prep folder
path = "data/roadnet/xml_prep/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)

# Clean up the xml_consolidated folder
path = "data/roadnet/xml_consolidated/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)    

### Convert D365 XML outbound files from a single line (with no newlines) to multiple lines by inserting a newline after evert '>' character

In [177]:
# Get the list of all files and directories
path = "data/roadnet/xml_outbound/"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

# prints all files
print(dir_list)

Files and directories in ' data/roadnet/xml_outbound/ ' :
['INT_LEW002_SendDataToRoadnet_EXP-Roadnet session composite entity. (7) (1).xml']


In [178]:
def xml_to_multiple_lines(fname, fnum):

    with open(fname, 'r') as fr:
        # reading line by line
        lines = fr.readlines()
        last_line = len(lines)
        print(last_line)

    for line in lines:
        replaced_line = re.sub(">", ">\u000A", line)

    outfile = 'data/roadnet/xml_prep/f'+str(fnum)+'.xml'

    with open(outfile, 'w') as fw:
        fw.write(replaced_line)    

    return

In [179]:
# Convert xml from D365 by adding newlines

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    print(xml_file)
    xml_to_multiple_lines(xml_file,i)

number_of_xml = len(dir_list)

data/roadnet/xml_outbound/INT_LEW002_SendDataToRoadnet_EXP-Roadnet session composite entity. (7) (1).xml
1


### Now remove all the lines starting with <CCBROADNETWORKBENCHSESSIONTABLEENTITY or </CCBROADNETWORKBENCHSESSIONTABLEENTITY

In [180]:
# Get the list of all files and directories
path = "data/roadnet/xml_prep/"
prep_dir_list = os.listdir(path)

In [181]:
def import_roadnet_files2(fname, fnum, outfile):

    try:    
        with open(fname, 'r') as fr:
            # reading line by line
            lines = fr.readlines()

            last_line = len(lines)

            # opening in writing mode
            with open(outfile, 'a') as fw:
                for line in lines:      
                    substring = 'CCBROADNETWORKBENCHSESSIONTABLEENTITY'       
                    x = re.search(substring, line)
                    #print(x)
                    if x == None:
                        fw.write(line)
                        #print(line)
        print(fname+" lines deleted")

    except:
        print("Error importing "+fname)

In [182]:
# Consolidate the xml files

for i in range(0,len(prep_dir_list)):
    xml_file = path+prep_dir_list[i]
    #print(i)
    print(xml_file)
    import_roadnet_files2(xml_file,i, consolidated_roadnet_out_file)

data/roadnet/xml_prep/f0.xml
data/roadnet/xml_prep/f0.xml lines deleted


### Split up the file in data/roadnet/xml_prep into 10,000 lines or less

In [183]:
fname = consolidated_roadnet_out_file
outfile = fname

try:    
    with open(fname, 'r') as fr:
        # reading line by line
        lines = fr.readlines()

        last_line = len(lines)

        line_counter = 1

        # opening in writing mode
        last_x = 0
        for i in range(0,len(lines)):
            line = lines[i]
            if i == last_line:
                print(line)
            x = int(i/10000)
            with open(outfile[:-4]+str(x)+'.xml', 'a') as fw:        
                if x > last_x:
                    last_x = x
                    fw.write('<?xml version="1.0" encoding="utf-8"?>\n')
                    fw.write('<Document>\n')
                fw.write(line)
                if int((i+1)/10000) > x:
                    fw.write('</Document>')
    os.remove(consolidated_roadnet_out_file)
                
except:
    print("Error importing "+fname)

### Rest starts here

In [184]:
# Get the list of all files and directories
path = "data/roadnet/xml_consolidated/"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

# prints all files
print(dir_list)

Files and directories in ' data/roadnet/xml_consolidated/ ' :
['consolidated_roadnet_out0.xml', 'consolidated_roadnet_out1.xml', 'consolidated_roadnet_out2.xml']


In [185]:
rdnet_out = pd.read_xml(path+dir_list[0])

In [186]:
for i in range(1,len(dir_list)):
    xml_file = path+dir_list[i]
    print(str(i)+xml_file)
    temp = pd.read_xml(path+dir_list[i])
    rdnet_out = pd.concat([rdnet_out, temp], ignore_index=True)

1data/roadnet/xml_consolidated/consolidated_roadnet_out1.xml
2data/roadnet/xml_consolidated/consolidated_roadnet_out2.xml


In [187]:
rdnet_out

,INVENTTRANSID,BLOCKEDSTATUS,CITY,CREDITRELEASEDATE,CREDITRELEASETIME,DAILYURGENCYINCREASE,DEPENDENCY,DESCRIPTION,DYNAMICSINTERNALSESSIONID,ERROR,...,REFTABLEID,ROADNETROUTE,SALESMANAGER,SHIPDATE,URGENCY,VOLUME,WAREHOUSEDESC,WAREHOUSEID,WEIGHT,WEIGHTUNIT
0,ZA1-006167875,0,Prieska,1900-01-01T00:00:00+00:00,0,0,0,USAVE PRIESKA,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,995471040.0,Gutsche Finished Goods WH,ZA027B,568.500,CS
1,ZA1-006549669,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,45603160.0,Gutsche Finished Goods WH,ZA027B,25.520,CS
2,ZA1-006550394,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,114007900.0,Gutsche Finished Goods WH,ZA027B,65.200,CS
3,ZA1-006550398,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,623973960.0,Gutsche Finished Goods WH,ZA027B,235.800,CS
4,ZA1-006550414,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,154260480.0,Gutsche Finished Goods WH,ZA027B,91.250,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26608,ZA1-007165386,0,Kimberley,1900-01-01T00:00:00+00:00,0,0,0,FISHAWAYS KIMBERLEY CBD,ZA1-000001374,NaN,...,1328,ZA532B01,NaN,2023-01-06T00:00:00+00:00,0,544268340.0,LDP Vaal Depot WH,ZA572B,309.650,CS
26609,ZA1-007148750,0,Zastron,1900-01-01T00:00:00+00:00,0,0,0,DIY DEPOT ZASTRON,ZA1-000001375,NaN,...,1328,ZA027B02,NaN,2023-01-10T00:00:00+00:00,0,271674000.0,LDP MAFIKENG WH,ZA537B,188.550,CS
26610,ZA1-007165387,0,Bloemfontein,1900-01-01T00:00:00+00:00,0,0,0,MANHUBE TUMELO PROJECTS,ZA1-000001377,NaN,...,1328,ZA531B01,NaN,2023-01-10T00:00:00+00:00,0,142545000.0,LDP MAFIKENG WH,ZA537B,77.292,CS
26611,ZA1-007148741,0,Bloemfontein,1900-01-01T00:00:00+00:00,0,0,0,2 FAT COWS,ZA1-000001378,NaN,...,1328,ZA531B01,NaN,2023-01-10T00:00:00+00:00,0,150000000.0,LDP BLOEMFONTEIN WH,ZA531B,80.250,CS


In [188]:
rdnet_in = rdnet_out[['QUANTITY','LOCATIONID','INVENTTRANSID','ITEMID','ORDERID','WAREHOUSEID','PRODUCTNAME','ROADNETROUTE','ORDERACCOUNT','ORDERACCOUNTNAME','WEIGHT']]

In [189]:
rdnet_in.rename(columns={'QUANTITY':'CASEQTY','LOCATIONID':'DESTINATIONLOCATIONID','ORDERID':'ORDERNUMBER','WAREHOUSEID':'ORIGINLOCATIONID','ORDERACCOUNT':'STOPLOCATIONID','ORDERACCOUNTNAME':'STOPLOCATIONNAME'}, inplace=True)

/tmp/ipykernel_322/554851200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdnet_in.rename(columns={'QUANTITY':'CASEQTY','LOCATIONID':'DESTINATIONLOCATIONID','ORDERID':'ORDERNUMBER','WAREHOUSEID':'ORIGINLOCATIONID','ORDERACCOUNT':'STOPLOCATIONID','ORDERACCOUNTNAME':'STOPLOCATIONNAME'}, inplace=True)


In [190]:
rdnet_in['DYNAMICSRETRIEVALSESSIONID'] = 'ZA1-000000661'

/tmp/ipykernel_322/3570605316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdnet_in['DYNAMICSRETRIEVALSESSIONID'] = 'ZA1-000000661'


In [191]:
today = str(datetime.now())
today = today.replace(':','h')
today = today.replace('-','')
today = today.replace(' ','-')
today = today[0:14] + '-'
print("Today date is: ", today)

Today date is:  20230113-15h30-


In [192]:
rdnet_in['ROADNETROUTEINTERNALROUTEID'] = today + rdnet_in['STOPLOCATIONID'].astype(str)

/tmp/ipykernel_322/1341196141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdnet_in['ROADNETROUTEINTERNALROUTEID'] = today + rdnet_in['STOPLOCATIONID'].astype(str)


In [193]:
no_of_customers = len(rdnet_in['STOPLOCATIONID'].unique())

In [194]:
rdnet_in['APPTID'] = ''
rdnet_in['DESCRIPTION'] = 'BLOEM_PLAN'
rdnet_in['ERROR'] = ''
rdnet_in['FIRSTDRIVER'] = '825196'
rdnet_in['FIRSTTRAILER'] = 'ST29PTAIL'
rdnet_in['LASTSTOPISDESTINATION'] = 'No'
rdnet_in['LOADID'] = ''
rdnet_in['LOADTEMPLATEID'] = ''
rdnet_in['ORDERTYPE'] = 'rotOrder'
rdnet_in['ORIGINDESTINATION'] = 'Yes'
rdnet_in['PALLETQTY'] = '0'
rdnet_in['REFERENCECATEGORY'] = 'Sales'
rdnet_in['REFERENCEDOCUMENT'] = 'SalesOrder'
rdnet_in['ROADNETINTERNALSESSIONID'] = '35411'
rdnet_in['ROADNETREGIONID'] = 'ZA1'
rdnet_in['ROUTECODE'] = ''
rdnet_in['SECONDDRIVER'] = ''
rdnet_in['SECONDTRAILER'] = ''
rdnet_in['SEQUENCEDISTANCE'] = '.000000'
rdnet_in['SEQUENCENUMBER'] = '1'
rdnet_in['SEQUENCETRAVELTIME'] = '0'
rdnet_in['SHIPPINGCARRIER'] = '0'
rdnet_in['STATUS'] = 'Error'
rdnet_in['STOPTYPE'] = 'stpStop'
rdnet_in['TOTALDISTANCE'] = '.000000'
rdnet_in['TOTALROUTEDISTANCE'] = '.000000'
rdnet_in['TRUCKANDTRAILERASSIGNED'] = 'No'
rdnet_in['UNITID'] = ''
rdnet_in['VEHICLEID'] = 'TT4X2TAIL'
rdnet_in['STOPSERVICETIME'] = '720'
rdnet_in['TOTALSERVICETIME'] = '720'
rdnet_in['TOTALTRAVELTIME'] = '0'
rdnet_in['LINEREFID'] = rdnet_in['INVENTTRANSID']


/tmp/ipykernel_322/1580939690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdnet_in['APPTID'] = ''
/tmp/ipykernel_322/1580939690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdnet_in['DESCRIPTION'] = 'BLOEM_PLAN'
/tmp/ipykernel_322/1580939690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [195]:
#rdnet_in.info()

In [196]:
rdnet_in['ROUTECOMPLETETIME'] = pd.to_datetime('today')
rdnet_in['ROUTECOMPLETETIME'] = rdnet_in['ROUTECOMPLETETIME'].dt.normalize() + pd.Timedelta(days=1) + pd.Timedelta(hours=13) + pd.Timedelta(minutes=19)

rdnet_in['ROUTESTARTTIME'] = pd.to_datetime('today')
rdnet_in['ROUTESTARTTIME'] = rdnet_in['ROUTESTARTTIME'].dt.normalize() + pd.Timedelta(days=1) + pd.Timedelta(hours=4) + pd.Timedelta(minutes=0)

rdnet_in['SCHEDULEDARRIVALDATETIME'] = pd.to_datetime('today')
rdnet_in['SCHEDULEDARRIVALDATETIME'] = rdnet_in['SCHEDULEDARRIVALDATETIME'].dt.normalize() + pd.Timedelta(days=1) + pd.Timedelta(hours=12) + pd.Timedelta(minutes=59)

rdnet_in['SCHEDULEDDELIVERYDATETIME'] = pd.to_datetime('today')
rdnet_in['SCHEDULEDDELIVERYDATETIME'] = rdnet_in['SCHEDULEDDELIVERYDATETIME'].dt.normalize() + pd.Timedelta(days=1)

rdnet_in['SCHEDULEDSHIPDATETIME'] = pd.to_datetime('today')
rdnet_in['SCHEDULEDSHIPDATETIME'] = rdnet_in['SCHEDULEDSHIPDATETIME'].dt.normalize() + pd.Timedelta(days=1) + pd.Timedelta(hours=4) + pd.Timedelta(minutes=10)

rdnet_in['STOPARRIVALTIME'] = pd.to_datetime('today')
rdnet_in['STOPARRIVALTIME'] = rdnet_in['STOPARRIVALTIME'].dt.normalize() + pd.Timedelta(days=1) + pd.Timedelta(hours=8) + pd.Timedelta(minutes=28)

### Get customer master in order to get the postal code

In [197]:
customers=pd.read_csv('data/customer_master.csv')
customers_short = customers[['ADDRESSZIPCODE','CUSTOMERACCOUNT','ORGANIZATIONNAME']].copy()
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].fillna(0)
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].astype(int)
customers_short['ADDRESSZIPCODE'] = customers_short['ADDRESSZIPCODE'].astype(str)

/tmp/ipykernel_322/2590905245.py:1: DtypeWarning: Columns (6,18,58,95,104,117,148) have mixed types. Specify dtype option on import or set low_memory=False.
  customers=pd.read_csv('data/customer_master.csv')


In [198]:
rdnet_in = pd.merge(
    rdnet_in,
    customers_short,
    how="inner",
    on=None,
    left_on='STOPLOCATIONID',
    right_on='CUSTOMERACCOUNT',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [199]:
rdnet_in.rename(columns={'ADDRESSZIPCODE':'STOPPOSTALCODE'}, inplace=True)
rdnet_in.drop(columns={'CUSTOMERACCOUNT', 'ORGANIZATIONNAME'}, inplace=True, axis=1)

In [200]:
rdnet_in.to_csv(roadnet_inbound_file,index=False)

In [211]:
rdnet_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26613 entries, 0 to 26612
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   INVENTTRANSID              26613 non-null  object 
 1   BLOCKEDSTATUS              26613 non-null  int64  
 2   CITY                       26613 non-null  object 
 3   CREDITRELEASEDATE          26613 non-null  object 
 4   CREDITRELEASETIME          26613 non-null  int64  
 5   DAILYURGENCYINCREASE       26613 non-null  int64  
 6   DEPENDENCY                 26613 non-null  int64  
 7   DESCRIPTION                26613 non-null  object 
 8   DYNAMICSINTERNALSESSIONID  26613 non-null  object 
 9   ERROR                      0 non-null      float64
 10  ITEMID                     26613 non-null  object 
 11  LOCATIONID                 26613 non-null  object 
 12  LOCATIONTYPE               26613 non-null  object 
 13  LOCSUBCHANNEL              24691 non-null  obj

In [201]:
rdnet_in

,CASEQTY,DESTINATIONLOCATIONID,INVENTTRANSID,ITEMID,ORDERNUMBER,ORIGINLOCATIONID,PRODUCTNAME,ROADNETROUTE,STOPLOCATIONID,STOPLOCATIONNAME,...,TOTALSERVICETIME,TOTALTRAVELTIME,LINEREFID,ROUTECOMPLETETIME,ROUTESTARTTIME,SCHEDULEDARRIVALDATETIME,SCHEDULEDDELIVERYDATETIME,SCHEDULEDSHIPDATETIME,STOPARRIVALTIME,STOPPOSTALCODE
0,12.0,10004160,ZA1-006562727,3040,1000054064,ZA027B,FANTA ORANGE 01X06 2000 SHR,ZA027B02,10004160.0,CHECKERS PRELLER PLAIN,...,720,0,ZA1-006562727,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9301
1,1.0,10004160,ZA1-006562748,7380,1000054064,ZA027B,POWERADE ORANGE 04X06 500 PET,ZA027B02,10004160.0,CHECKERS PRELLER PLAIN,...,720,0,ZA1-006562748,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9301
2,1.0,10004160,ZA1-006562755,5821,1000054064,ZA027B,SCHW LEMONADE 04X06 200 CAN,ZA027B02,10004160.0,CHECKERS PRELLER PLAIN,...,720,0,ZA1-006562755,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9301
3,3.0,10004160,ZA1-006562756,5491,1000054064,ZA027B,SCHW TONIC 01X12 1000 SHR,ZA027B02,10004160.0,CHECKERS PRELLER PLAIN,...,720,0,ZA1-006562756,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9301
4,2.0,10004160,ZA1-006562777,7915,1000054064,ZA027B,VALPRE STILL 01X12 1500 PET,ZA027B02,10004160.0,CHECKERS PRELLER PLAIN,...,720,0,ZA1-006562777,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25441,1.0,10132102,ZA1-006884688,5391,1000056208,ZA027B,SCHW SODA WATER 01X12 1000 SHR,ZA027B02,10132102.0,SPAR TOPS HENNENMAN,...,720,0,ZA1-006884688,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9445
25442,2.0,10132102,ZA1-006884701,1091,1000056208,ZA027B,COKE 01X12 1000 SHR,ZA027B02,10132102.0,SPAR TOPS HENNENMAN,...,720,0,ZA1-006884701,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9445
25443,2.0,10132102,ZA1-006884703,5991,1000056208,ZA027B,SCHW GINGERALE 01X12 1000 SHR,ZA027B02,10132102.0,SPAR TOPS HENNENMAN,...,720,0,ZA1-006884703,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9445
25444,1.0,10132102,ZA1-006884704,5491,1000056208,ZA027B,SCHW TONIC 01X12 1000 SHR,ZA027B02,10132102.0,SPAR TOPS HENNENMAN,...,720,0,ZA1-006884704,2023-01-14 13:19:00,2023-01-14 04:00:00,2023-01-14 12:59:00,2023-01-14,2023-01-14 04:10:00,2023-01-14 08:28:00,9445


In [202]:
#rdnet_out_cons = rdnet_out
rdnet_out_cons = pd.concat([rdnet_out_cons, rdnet_out], ignore_index=True)

In [217]:
duplicateRows = rdnet_out_cons[rdnet_out_cons.duplicated(['INVENTTRANSID','DYNAMICSINTERNALSESSIONID','ITEMID','ORDERACCOUNT'])]

In [218]:
duplicateRows

,INVENTTRANSID,BLOCKEDSTATUS,CITY,CREDITRELEASEDATE,CREDITRELEASETIME,DAILYURGENCYINCREASE,DEPENDENCY,DESCRIPTION,DYNAMICSINTERNALSESSIONID,ERROR,...,REFTABLEID,ROADNETROUTE,SALESMANAGER,SHIPDATE,URGENCY,VOLUME,WAREHOUSEDESC,WAREHOUSEID,WEIGHT,WEIGHTUNIT
26990,ZA1-006167875,0,Prieska,1900-01-01T00:00:00+00:00,0,0,0,USAVE PRIESKA,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,995471040.0,Gutsche Finished Goods WH,ZA027B,568.500,CS
26991,ZA1-006549669,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,45603160.0,Gutsche Finished Goods WH,ZA027B,25.520,CS
26992,ZA1-006550394,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,114007900.0,Gutsche Finished Goods WH,ZA027B,65.200,CS
26993,ZA1-006550398,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,623973960.0,Gutsche Finished Goods WH,ZA027B,235.800,CS
26994,ZA1-006550414,0,Welkom,1900-01-01T00:00:00+00:00,0,0,0,GAME WELKOM 163,ZA1-000001227,NaN,...,1328,ZA027B02,NaN,2022-10-24T00:00:00+00:00,0,154260480.0,Gutsche Finished Goods WH,ZA027B,91.250,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53598,ZA1-007165386,0,Kimberley,1900-01-01T00:00:00+00:00,0,0,0,FISHAWAYS KIMBERLEY CBD,ZA1-000001374,NaN,...,1328,ZA532B01,NaN,2023-01-06T00:00:00+00:00,0,544268340.0,LDP Vaal Depot WH,ZA572B,309.650,CS
53599,ZA1-007148750,0,Zastron,1900-01-01T00:00:00+00:00,0,0,0,DIY DEPOT ZASTRON,ZA1-000001375,NaN,...,1328,ZA027B02,NaN,2023-01-10T00:00:00+00:00,0,271674000.0,LDP MAFIKENG WH,ZA537B,188.550,CS
53600,ZA1-007165387,0,Bloemfontein,1900-01-01T00:00:00+00:00,0,0,0,MANHUBE TUMELO PROJECTS,ZA1-000001377,NaN,...,1328,ZA531B01,NaN,2023-01-10T00:00:00+00:00,0,142545000.0,LDP MAFIKENG WH,ZA537B,77.292,CS
53601,ZA1-007148741,0,Bloemfontein,1900-01-01T00:00:00+00:00,0,0,0,2 FAT COWS,ZA1-000001378,NaN,...,1328,ZA531B01,NaN,2023-01-10T00:00:00+00:00,0,150000000.0,LDP BLOEMFONTEIN WH,ZA531B,80.250,CS


In [220]:
no_dup = rdnet_out_cons.drop_duplicates(keep=False)

In [221]:
no_dup

,INVENTTRANSID,BLOCKEDSTATUS,CITY,CREDITRELEASEDATE,CREDITRELEASETIME,DAILYURGENCYINCREASE,DEPENDENCY,DESCRIPTION,DYNAMICSINTERNALSESSIONID,ERROR,...,REFTABLEID,ROADNETROUTE,SALESMANAGER,SHIPDATE,URGENCY,VOLUME,WAREHOUSEDESC,WAREHOUSEID,WEIGHT,WEIGHTUNIT
26613,ZA1-007175425,0,Douglas,1900-01-01T00:00:00+00:00,0,0,0,SPAR DOUGLAS,ZA1-000001380,NaN,...,1328,ZA027B02,NaN,2023-01-12T00:00:00+00:00,0,5.937473e+10,Pretoria Finished Goods WH,ZA014B,33780.000,CS
26614,ZA1-007175411,0,Northern Cape,1900-01-01T00:00:00+00:00,0,0,0,SPAR WARRENTON,ZA1-000001381,NaN,...,1328,ZA027B02,NaN,2023-01-12T00:00:00+00:00,0,2.849987e+10,Premier Finished Goods WH,ZA036B,16571.520,CS
26615,ZA1-007148743,0,Vryburg,1900-01-01T00:00:00+00:00,0,0,0,JOHANEY TUCK SHOP,ZA1-000001382,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,4.751500e+07,Premier Finished Goods WH,ZA036B,25.764,CS
26616,ZA1-007148744,0,Vryburg,1900-01-01T00:00:00+00:00,0,0,0,JOHANEY TUCK SHOP,ZA1-000001382,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,2.100000e+08,Premier Finished Goods WH,ZA036B,112.350,CS
26617,ZA1-007148745,0,Vryburg,1900-01-01T00:00:00+00:00,0,0,0,JOHANEY TUCK SHOP,ZA1-000001382,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,1.086405e+07,Premier Finished Goods WH,ZA036B,7.350,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26985,ZA1-007176516,0,Olifantshoek,1900-01-01T00:00:00+00:00,0,0,0,LUCKY SHOP,ZA1-000001395,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,3.318237e+07,LDP OSIZWENI WH,ZA508B,18.740,CS
26986,ZA1-007176517,0,Olifantshoek,1900-01-01T00:00:00+00:00,0,0,0,LUCKY SHOP,ZA1-000001395,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,2.142524e+07,LDP OSIZWENI WH,ZA508B,12.390,CS
26987,ZA1-007176520,0,Olifantshoek,1900-01-01T00:00:00+00:00,0,0,0,LUCKY SHOP,ZA1-000001395,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,2.079913e+07,LDP OSIZWENI WH,ZA508B,11.220,CS
26988,ZA1-007176340,0,Olifantshoek,1900-01-01T00:00:00+00:00,0,0,0,LUCKY SHOP,ZA1-000001395,NaN,...,1328,ZA533B01,NaN,2023-01-12T00:00:00+00:00,0,3.085210e+07,LDP OSIZWENI WH,ZA508B,18.250,CS
